# Miner Notebook : 02

This notebook is used for experimentation during the implementation of the Zimcoin miner component.

In [1]:
import binascii
import miner
from miner import ZimcoinMiner
from Library.opencl_information import opencl_information
from blocks import Block, mine_block
from transactions import Transaction

from importlib import reload

## Helper Functions to create test blocks 

In [2]:
def get_test_first_block() -> Block:
    """
    Get a block representing the first in a chain for testing.
    """
    return Block(
            bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000'),
            0,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [],
            1626626569,
            100000000,
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            55419)

def get_transactions_block() -> Block:
    """
    Get a block with transactions for testing.
    """
    return Block(
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            1,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [
                Transaction(
                    bytes.fromhex("9e09208d54c012c0844cf17cfbb175157516dc90"),
                    bytes.fromhex("4f3ea27a7af06cbe53911d4fb9326730d435255a"),
                    bytes.fromhex("3056301006072a8648ce3d020106052b8104000a034200041a719dc420fdbdeef447e90a6368b9486d4afbacd900f6d9d5f62692dfa9ecb695999af4fcf61bdc523021b3aef2b84344b7c4ba7d3a36efe2e5f3eff50e8c54"),
                    390,
                    5,
                    1,
                    bytes.fromhex("3045022100fae9ab97090f2f0fb5715497e12a06438cbccc610bae2f9c019dfa5bdb40f0090220283f5498f22e17ac9ecf4c239d864811dd47cb0ccb8c3584794791fd171e6b90"),
                    bytes.fromhex("0cfd04ed0b2b279c12412687c770b1224c8bfed453292652694339ddade4d63a")),
            ],
            1626626571,
            1000000000,
            bytes.fromhex('000071f1c701e06e5b91adb4289d6c5227b614bd4441748923826e5d0e8828da'),
            83651)

## Show the available OpenCL Platforms

In [3]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: NVIDIA CUDA
Platform 0 - Vendor: NVIDIA Corporation
Platform 0 - Version: OpenCL 3.0 CUDA 11.7.89
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: NVIDIA GeForce RTX 3060
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1867 Mhz
 Device - Compute Units: 28
 Device - Local Memory: 48 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 12 GB
 Device - Max Buffer/Image Size: 3011 MB
 Device - Max Work Group Size: 1024


Platform 1 - Name: Portable Computing Language
Platform 1 - Vendor: The pocl project
Platform 1 - Version: OpenCL 1.2 pocl 1.6, None+Asserts, LLVM 9.0.1, RELOC, SLEEF, DISTRO, POCL_DEBUG
Platform 1 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: pthread-AMD Ryzen 5 3600 6-Core Processor
 Device - Type: ALL | CPU
 Device - Max Clock Speed: 4208 Mhz
 Device - Compute Units: 12
 Device - Local Memory: 512 K

## Create the Miner

In [4]:
reload(miner)

zimcoin_miner = ZimcoinMiner(0, 0)
print('Threads:', zimcoin_miner.thread_count)

Threads: 28672


In [5]:
def mine_block(miner : ZimcoinMiner, block : Block) -> int:
    """
    Mine a block.

    Parameters
    ----------
    block : Block

    Returns
    -------
    int
        The nonce for the block.
    """
    return miner.mine(
        block.previous,
        block.height,
        block.miner,
        block.transactions,
        block.timestamp,
        block.difficulty)

## Mine the first block

In [6]:
# calculate the nonce
first_block = get_test_first_block()
first_nonce = mine_block(zimcoin_miner, first_block)

# update the block with the nonce
first_block.nonce = first_nonce

print('Nonce    :', first_nonce)
print('Target   :', binascii.hexlify(first_block.calculate_target().to_bytes(32, 'big')))
print('Block ID :', binascii.hexlify(first_block.calculate_block_id()))


Nonce    : 22273193404
Target   : b'0000002af31dc4611873bf3f70834acdae9f0f4f534f5d60585a5f1c1a3ced1b'
Block ID : b'00000019a10edf91e1d75eec93991ba7f5cdb2ae0646bcd9958e08664181a1ba'


## Mine the second block

In [7]:
# calculate the nonce
second_block = get_transactions_block()
second_nonce = mine_block(zimcoin_miner, second_block)

# update the block with the nonce
second_block.nonce = second_nonce

print('Nonce    :', second_nonce)
print('Target   :', binascii.hexlify(second_block.calculate_target().to_bytes(32, 'big')))
print('Block ID :', binascii.hexlify(second_block.calculate_block_id()))


Nonce    : 26137320674
Target   : b'000000044b82fa09b5a52cb98b405447c4a98187eebb22f008d5d64f9c394ae9'
Block ID : b'00000001e728e0248875cafb767fa5caaf1d9ee625c870718897fd95c70a0885'
